In [1]:
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
from itertools import product

seqs = [''.join(c) for c in product('ATCG', repeat=8)]

rdict = {'A':'T', 'T':'A','C':'G', 'G':'C'}
def rc (seq):
    return "".join([rdict[c]for c in seq][::-1])

forwards = []
for s in seqs:
    if rc(s) in forwards:
        pass
    else:
        forwards.append(s)
len(forwards)
forwards = list(
    filter(
    lambda seq: 15.5 <= mt.Tm_NN(seq, Mg = 11.5, Na = 5, saltcorr=4) <= 16.5,
        forwards
    )
)
len(forwards)
#CG contents
forwards = [
    seq for seq in forwards
    if (seq.count("C") + seq.count("G")) / len(seq) == 0.5
]
len(forwards)

1227

In [ ]:
import random
from nupack import *
from tqdm import tqdm

model1 = Model(material='dna', celsius=16, sodium=0.05, magnesium=0.00)
max_unique_seq = []
max_len = 0


for r in tqdm(range(len(forwards))):
    unique_seq = []
    potential_seq = []
    if len(unique_seq) == 0:
        unique_seq.append(forwards[r])
        unique_seq.append(rc(forwards[r]))


    for r1 in range(len(forwards)):    
        count = 0
        for b, uni_seq in enumerate(unique_seq):
            if b % 2 ==0: #only want to loop the forward seq
                if r1 != r:
                    potential_seq.append(forwards[r1])

                    dG_comp_pot = pfunc((forwards[r1], rc(forwards[r1])), model=model1)

                    dG_pot_f_1 = pfunc((forwards[r1], uni_seq), model = model1) #compare dG from possible pair (forward) to the sequences inside unique_seq

                    dG_pot_f_2 = pfunc((forwards[r1], rc(uni_seq)), model = model1)

                    dG_pot_r_1 = pfunc((rc(forwards[r1]),uni_seq),model=model1) #compare dG from possible pair (reverse) to the sequences inside unique_seq

                    dG_pot_r_2 = pfunc((rc(forwards[r1]),rc(uni_seq)), model=model1)

                    dG_palindromic = pfunc((forwards[r1],forwards[r1]), model=model1)

                    ddG_pot_f_1 = dG_comp_pot[1] - dG_pot_f_1[1]
                    ddG_pot_f_2 = dG_comp_pot[1] - dG_pot_f_2[1]

                    ddG_pot_r_1 = dG_comp_pot[1] - dG_pot_r_1[1]
                    ddG_pot_r_2 = dG_comp_pot[1] - dG_pot_r_2[1]

                    ddG_palindromic = dG_comp_pot[1] - dG_palindromic[1]

                    if ddG_pot_f_1 <= -4.1 and ddG_pot_f_2 <= -4.1 and ddG_pot_r_1 <= -4.1 and ddG_pot_r_2 <= -4.1 and ddG_palindromic <= -4: #ddG_pal maybe bs
                        # print(ddG_pot_f_1,',',ddG_pot_f_2,',',ddG_pot_r_1,',',ddG_pot_r_2)
                        count += 2
                    # print(count)
                    if count == len(unique_seq): #why unique_seq cannot more than 4
                        # print(len(unique_seq))

                        count_1 = 0
                        for c, strand in enumerate(unique_seq):

                            if c % 2 ==0:
                                dG_comp_1 = pfunc((strand, rc(strand)), model=model1) #compare dG from the sequences inside unique_seq to the possible pair

                                dG_forw_1 = pfunc((strand,potential_seq[0]), model=model1) #check forward strand inside unique with forward and reverse in the possible pair
                                dG_forw_2 = pfunc((strand,rc(potential_seq[0])),model=model1)


                                dG_rev_1 = pfunc((rc(strand),potential_seq[0]), model=model1) #check reverse strand inside unique with forward and reverse in the possible pair
                                dG_rev_2 = pfunc((rc(strand),rc(forwards[r1])),model=model1)



                                ddG_forw_1 = dG_comp_1[1] - dG_forw_1[1]
                                ddG_forw_2 = dG_comp_1[1] - dG_forw_2[1]


                                ddG_rev_1 = dG_comp_1[1] - dG_rev_1[1]
                                ddG_rev_2 = dG_comp_1[1] - dG_rev_2[1]



                                if ddG_forw_1 <= -4.1 and ddG_forw_2 <= -4.1 and ddG_rev_1 <= -4.1 and ddG_rev_2 <= -4.1:
                                    # print(ddG_forw_1,',',ddG_forw_2,',',ddG_rev_1,',',ddG_rev_2)
                                    count_1 += 2   #why always = 2

                                if count_1 == len(unique_seq):

                                    unique_seq.append(potential_seq[0])
                                    unique_seq.append(rc(potential_seq[0]))


                    potential_seq = []


    if len(unique_seq) > max_len or len(unique_seq) >= 48:
        max_len = len(unique_seq)
        max_unique_seq = unique_seq[:]
        for i in range(0, len(max_unique_seq), 2):
            print(max_unique_seq[i])
            print(max_unique_seq[i+1])
        print(len(max_unique_seq))  

  0%|                                              | 1/1227 [01:10<24:01:16, 70.54s/it]

AAAAGCCC
GGGCTTTT
AAATCGGG
CCCGATTT
AAACACCC
GGGTGTTT
AAACCTCG
CGAGGTTT
AAAGGACG
CGTCCTTT
AATCCACG
CGTGGATT
AATGCTGG
CCAGCATT
AATGGTCG
CGACCATT
AACTAGCG
CGCTAGTT
AACTCTGC
GCAGAGTT
AACGAAGG
CCTTCGTT
AAGTGAGC
GCTCACTT
ACCTGACA
TGTCAGGT
AGACAAGC
GCTTGTCT
AGGAAACG
CGTTTCCT
TTGTTGGG
CCCAACAA
TCATTGCC
GGCAATGA
TCTTACGC
GCGTAAGA
TCTGTTCG
CGAACAGA
CAGTTTGC
GCAAACTG
40


  0%|                                              | 2/1227 [02:34<26:40:28, 78.39s/it]

AAAAGGCC
GGCCTTTT
AAATCCCG
CGGGATTT
AAACTCCG
CGGAGTTT
AAACCACC
GGTGGTTT
AAACGAGG
CCTCGTTT
AAAGACCG
CGGTCTTT
AAAGTCGG
CCGACTTT
AAAGCAGG
CCTGCTTT
AATAGCGG
CCGCTATT
AATGACGG
CCGTCATT
AATGTCCG
CGGACATT
AACAGAGC
GCTCTGTT
AACTTCGG
CCGAAGTT
AAGGAACG
CGTTCCTT
ATGATGGC
GCCATCAT
ACATTCCG
CGGAATGT
ACCAATCG
CGATTGGT
AGCACTTC
GAAGTGCT
TTCTCACG
CGTGAGAA
TCTTTGGC
GCCAAAGA
TGCAAGAG
CTCTTGCA
42


  0%|                                              | 3/1227 [03:50<26:13:27, 77.13s/it]

...

 71%|█████████████████████████████▋            | 866/1227 [17:27:19<7:47:14, 77.66s/it]

TCAATCCG
CGGATTGA
AAAAGCCC
GGGCTTTT
AAACACCC
GGGTGTTT
AAACTCGG
CCGAGTTT
AAAGACGG
CCGTCTTT
AAAGTCCG
CGGACTTT
AAAGGTCG
CGACCTTT
AATAGCGG
CCGCTATT
AATCGTGG
CCACGATT
AATGCTGG
CCAGCATT
AACTTCCG
CGGAAGTT
AACTGAGC
GCTCAGTT
AACCAACC
GGTTGGTT
AAGAGTGC
GCACTCTT
ATTTCCCG
CGGGAAAT
ATCCATGC
GCATGGAT
ATGAACCG
CGGTTCAT
ACATTCGG
CCGAATGT
ACGTTACC
GGTAACGT
AGACAAGC
GCTTGTCT
TCATTGCC
GGCAATGA
TCGAAACC
GGTTTCGA
TGTCATCG
CGATGACA
CTGTTTGC
GCAAACAG
48


 71%|█████████████████████████████▋            | 867/1227 [17:28:44<7:59:35, 79.93s/it]

TCAATCGG
CCGATTGA
AAAAGCCC
GGGCTTTT
AAACACCC
GGGTGTTT
AAACCTCG
CGAGGTTT
AAAGACGG
CCGTCTTT
AAAGTCCG
CGGACTTT
AATAGCGG
CCGCTATT
AATCCACG
CGTGGATT
AATGACCG
CGGTCATT
AATGTCGG
CCGACATT
AATGCTGG
CCAGCATT
AACTTCCG
CGGAAGTT
AACTCTGC
GCAGAGTT
AAGTGAGC
GCTCACTT
AAGGAACG
CGTTCCTT
ATTTCCCG
CGGGAAAT
ACTTTCGG
CCGAAAGT
AGACAAGC
GCTTGTCT
TTTCAGGC
GCCTGAAA
TTGTTGGG
CCCAACAA
TCAATGGC
GCCATTGA
TCTTTGCC
GGCAAAGA
TCGTATCG
CGATACGA
CGTTTAGC
GCTAAACG
48


 72%|██████████████████████████████            | 878/1227 [17:41:42<7:02:09, 72.58s/it]

In [ ]:
### Check dGs for spurious strands ###

from nupack import *
import nupack
model1 = Model(material='dna', celsius=16, magnesium=0.00, sodium=0.05) #salts unit here is in M, we want mM 10mM Mg, 0 mM Na (Range:Na [0.05,1.1] ; Mg [0.0,0.2])

strands = """
AATCAGGC
GCCTGATT
TAGTTGCC
""".split()

sequences = """
AATCAGGC
GCCTGATT
TAGTTGCC
""".split()

def reverse_compliment(seq):
    seq_dict = {'A':'T', 'T':'A','C':'G', 'G':'C'}
    reverse_compliment = ''
    for base in seq[::-1]:
        if base == ' ': #if block to clean up the spacing in the data above, if empty space then add an empty space
            reverse_compliment += ' '
        else: #if not empty space look up the dictionary
            reverse_compliment += seq_dict[base]
    return reverse_compliment

dGs = []
for strd in strands:
    for seq in sequences:
        # if seq == strd: #or seq == reverse_compliment(strd):
        #     continue
        pair = [strd, seq]
        result = mfe(strands=[strd, seq], model=model1)
        # sampled_structures = sample(strands=[strd, seq], num_sample=2, model=model1)
        dGs.append([result[0],strd, seq ]) #append strand + sequence along with dG
        
# Print the ΔG values with orders and the strand + each sequence
for i, dG in enumerate(dGs, start = 1):
    print(f'{i}\t {dG[0]} \t {dG[1]}+{dG[2]}')
    print()

In [ ]:
### Check Probability Matrix of Spurious Strands ###
from nupack import *
import nupack
import matplotlib.pyplot as plt
import numpy as np
model1 = Model(material='dna', celsius=16, sodium=0.05, magnesium= 0.01)


a = 'CGTTGAAG'

strand_sequences = """
CGTTGAAG
CTTCAACG
ACACCATG
CATGGTGT
CATTCGAC
GTCGAATG
TAGCCACA
TGTGGCTA
CGTTGAAG
CTTCAACG
""".split()


def plot_probability_matrix(probability_matrix):
    plt.imshow(probability_matrix.to_array())
    plt.xlabel('Base index')
    plt.ylabel('Base index')
    plt.title('Pair probabilities for complex')
    plt.colorbar()
    plt.clim(0,1)
    plt.show()

for i, strand in enumerate(strand_sequences, start = 2):
    if strand == a or strand == reverse_complement(a):
        continue
    probability_matrix = pairs(strands=[a, strand], model=model1)
    mfe_result = mfe(strands=[a, strand], model=model1)
    sampled_structures = sample(strands=[a, strand], num_sample=2, model=model1)
    print(f'{i}. {a}+{strand}')
    print(sampled_structures)
    print(mfe_result[0])
    plot_probability_matrix(probability_matrix)